# AI Exploratory Data Analyst 📊

##### 💡 **Research Areas:** Rapid Prototyping, Generative AI, Exploratory Data Analysis, Question-Answering Systems.
<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="300px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>

An AI data analyst that performs **Exploratory Data Analysis** on a dataset using question-answering model with LLMs.

The goal of this prototype is to demonstrate the automation of Data Analytics Activity, highlight use-cases for Pandas AI, and experiment with Question-Answer type models.


### Step 1: Importing Dependencies

- `os`
Accesses environment variables and file paths.

- `pandas as pd`
Loads and manipulates data in tabular format.

- `load_dotenv`
Loads API keys securely from a `.env` file.

- `typing (List, Union)`
Provides type annotations.

- `SmartDataframe, SmartDatalake`
Adds AI-driven querying to Pandas.

- `OpenAI`
Enables natural language queries with OpenAI models.

- `BaseModel`
Ensures structured data validation.

- `Groq`
Connects to Groq AI for generating responses.

- `instructor`
Manages AI query execution efficiently.


### Step 2: Installing Requirements

The `install_requirements()` function ensures the environment is set up by installing dependencies from a requirements.txt file. It checks if dependencies are already installed to avoid redundancy and attempts installation up to three times if it fails. This guarantees that all necessary packages are available for the application to run smoothly.

In [45]:
import os

requirements_installed = False
max_retries = 3
retries = 0


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return
install_requirements()

## Step 3: Setting Up Environment Variables

This script loads and checks essential environment variables needed for the application. It uses `load_dotenv()` to load variables from the `.env` file, ensuring that any existing variables are overwritten. The `check_env()` function checks if each required variable (`GROQ_API_KEY`, `OPENAI_API_KEY`, `PANDASAI_API_KEY`) is set in the environment, and if not, the script exits with an error message. If all variables are set, the script confirms they are properly loaded.


In [48]:
from dotenv import load_dotenv
import os


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv(override=True)

    variables_to_check = ["GROQ_API_KEY", "OPENAI_API_KEY", "PANDASAI_API_KEY"]

    for var in variables_to_check:
        check_env(var)
        
setup_env()

## Step 4: Groq Client Setup

This setup initializes a Groq AI client using an API key from environment variables.


- `get_groq_client()`: Creates and returns a Groq client using instructor with JSON mode.

- `llm(prompt, response_model, system, model)`: Sends a user query to the Groq model, formats responses per `response_model`, and returns results.

- `Error Handling`: Catches exceptions, prints errors, and returns an `LLMErrorResponse`.

In [50]:
import instructor
from groq import Groq
import traceback
from pydantic import BaseModel
from typing import Union


def get_groq_client():
    """Returns an instance of the Groq class"""
    groq = Groq(api_key=os.getenv("GROQ_API_KEY"))
    client = instructor.from_groq(groq, mode=instructor.Mode.JSON)
    return client


def llm(
    prompt: str,
    response_model: BaseModel,
    system="You are a helpful AI assistant. The user will talk to you and its your job to provide detailed and clear responses.",
    model=DEFAULT_MODEL,
) -> Union[BaseModel, LLMErrorResponse]:
    """Calls LLM API with the given prompt. Defaults to llama-3.3-70b-versatile""",
    try:
        client = get_groq_client()

        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ]

        response = client.chat.completions.create(
            messages=messages, model=model, response_model=response_model
        )
        return response
    except Exception as e:
        traceback.print_exc()
        return LLMErrorResponse(error=str(e))
    
setup_env()

## Step 5: Dataset Querying and Analysis System

This code defines a class structure to load, manage, and query datasets, using an LLM to provide insights from data.


- The `Dataset` class initializes by loading tables from CSV files into a `SmartDataframe` and `SmartDatalake`. It uses the `OpenAI` API to facilitate querying the data with natural language.

- The `ask()` method lets you query the dataset, processes the response, and formats it according to the specified format (markdown by default).

- The `BooksDataset` and `StudentPerformanceDataset` classes are subclasses of `Dataset`, pre-configured for specific datasets, such as books or student performance.

- The `SmartDataframe` and `SmartDatalake` are used to enhance the data with AI capabilities for more insightful querying.



In [52]:
import pandas as pd
import os
from typing import List
from pandasai import SmartDataframe, SmartDatalake
from pandasai.llm import OpenAI


class FormattedLLMResponse(BaseModel):
    response: str


class Dataset:
    """Dataset class which contains tables corresponding to CSV files in the base folder"""

    def __init__(self, database: str, tables: List[str]):
        llm = OpenAI(api_token=os.getenv("OPENAI_API_KEY"))
        if len(tables) == 0:
            raise ValueError("Tables list cannot be empty.")

        if not os.path.isdir(f"data/{database}"):
            raise FileNotFoundError(f"Directory {database} not found.")

        self.base_path = os.path.join(os.getcwd() + "/data", database)
        self.tables = {}
        self.dataframes = {}

        for table in tables:
            table_file_path = os.path.join(self.base_path, f"{table}.csv")
            if not os.path.isfile(table_file_path):
                print(f"Table file {table_file_path} not found. Skipping. ")
                continue
            df = pd.read_csv(table_file_path)
            self.tables[table] = SmartDataframe(df)
            self.dataframes[table] = pd.read_csv(table_file_path)

        print(f"Loaded {len(self.tables)} tables from {database} dataset.")

        self.datalake = SmartDatalake(
            [self.tables[key] for key in self.tables.keys()], config={"llm": llm}
        )

    def get_table(self, table_name: str) -> SmartDataframe:
        table = self.tables.get(table_name)
        if table is None:
            raise ValueError(f"Table {table_name} not found.")
        return table

    def get_datalake(self) -> SmartDatalake:
        return self.datalake

    def ask(self, query: str, format="markdown") -> str:
        print("Asking...")
        datalake = self.get_datalake()
        df = datalake.chat(query)
        print("Asked.")
        df_string = ""
        if type(df) == str:
            df_string = df
        elif type(df) == pd.DataFrame:
            df_top = df.head(10)
            df_string = str(df_top)
        prompt = f"""
            Query: {query}
            Database Result: {df_string}
            Instructions:
            - Format the data and provide a human readable summary.
            - Provide insights and recommendations.
            - Don't hallucinate or make up data.
            - Provide clear, detailed and concise responses.
            f"The 'response' field should be of format = '{format}'" if format == "markdown" else ""
        """
        llm_response = llm(prompt, response_model=FormattedLLMResponse)
        return llm_response.response


class BooksDataset(Dataset):
    def __init__(self):
        super().__init__("books", ["books", "ratings", "users"])


class StudentPeformanceDataset(Dataset):
    def __init__(self):
        super().__init__("student_performance", ["student_performance"])

## Step 6: Render Markdown Output

This function, `render_output()`, takes a string containing markdown content and displays it as rendered Markdown in an IPython environment (e.g., Jupyter Notebook).

- It uses `Markdown` from `IPython.display` to convert the markdown text into a formatted output.

- This allows you to visualize the results of queries or insights in a human-readable markdown format within the notebook.

The function doesn't return a value; it just renders the formatted markdown.


In [53]:
from IPython.display import Markdown


def render_output(markdown: str) -> None:
    """Renders the generated output file as markdown."""
    return Markdown(markdown)

## Step 7: Querying Dataset and Displaying the Result

The ask_query function allows interactive querying of a dataset, ensuring clear and formatted output in a Jupyter notebook.


- The function takes a `query` and a `dataset` as input. It calls the `ask()` method from the dataset to get the query's response.

- `clear_output()` ensures the output area is cleared before showing the new response.

- The response is formatted by adding the query as a heading and is passed to the `render_output()` function to display the formatted result as Markdown in the notebook.

In [54]:
from IPython.display import clear_output


def ask_query(query: str, dataset: Dataset) -> None:
    """Asks the given query to the dataset and returns the response."""
    response = dataset.ask(query)
    clear_output()
    response = f"### Query: {query}\n{response}"
    return render_output(response)

## Step 8: Query Execution on Books and Student Performance

`ask_query_on_books` & `ask_query_on_student_performance`:

- Specialized functions that initialize the respective dataset objects and query the data using the ask_query method.

In [55]:
def ask_query_on_books(query: str) -> None:
    """Main function to run the script"""
    books_dataset = BooksDataset()
    return ask_query(query, books_dataset)


def ask_query_on_student_performance(query: str) -> None:
    """Main function to run the script"""
    student_performance_dataset = StudentPeformanceDataset()
    return ask_query(query, student_performance_dataset)

## Step 9: Running Queries
- Sample Queries:

    - Queries for top 10 books with highest average ratings and top 10 students with highest average grades.

In [ ]:
query = "What are the top 10 books with the highest average rating?"

ask_query_on_books(query)

query = "What are the top 10 students with the highest average grades?"

ask_query_on_student_performance(query)

## Step 10: DataAnalysisAgent Class

- Handles data analysis by asking a series of predefined questions and aggregating the answers into a business report.
- `analyse` method: Asks predefined questions, compiles answers, and generates a report using LLM.

- `get_analysis` method: Returns the analysis, either generated or cached.

In [58]:
from pydantic import BaseModel


class BusinessReport(BaseModel):
    markdown: str


class DataAnalysisAgent:
    def __init__(self, dataset: Dataset):
        self.dataset = dataset
        self.analysis = None

    def analyse(self) -> str:
        # instruction = "Respond in a human readable format. Don't provide graphs or execute code."
        analysis_questions = [
            "What are the key insights from the dataset?",
            "What are the key trends in the dataset?",
            "What are the key patterns in the dataset?",
            "What are the key anomalies in the dataset?",
            "What are the key recommendations based on the dataset?",
        ]
        self.analysis = ""
        for question in analysis_questions:
            try:
                answer = self.dataset.ask(question)
                self.analysis += f"\n### 👉🏽 {question}\n💡 {answer}"
            except Exception as e:
                print(f"Failed to ask question: {question}")
                # print(e)
                continue
        print("Preliminary analysis completed. Generating business report...")
        prompt = f"""
            Understand the provided analysis for the dataset and provide a neatly formatted report.
            This report should be professional, detailed and ideal for business stakeholders.
            The report should include all facts in the preliminary analysis and provide additional insights.
            The 'response' field should be of format = 'markdown'

            Preliminary Analysis: {self.analysis}
        """
        llm_response = llm(prompt=prompt, response_model=BusinessReport)
        if not llm_response:
            print("Failed to get response from LLM.")
            return self.analysis
        self.analysis = llm_response.markdown
        print("Business report generated.")
        return self.analysis

    def get_analysis(self) -> str:
        if self.analysis is None:
            return self.analyse()
        return self.analysis

## Step 11: Data Analysis Execution
- `data_analysis_on_books` & `data_analysis_on_student_performance`:

    - Calls the DataAnalysisAgent on the respective datasets (Books and Student Performance) and renders the analysis.

In [59]:
from IPython.display import clear_output, Markdown


def data_analysis(dataset: Dataset) -> Markdown:
    agent = DataAnalysisAgent(dataset)
    analysis = agent.analyse()
    clear_output()
    return render_output(analysis)


def data_analysis_on_books() -> Markdown:
    books_dataset = BooksDataset()
    return data_analysis(books_dataset)


def data_analysis_on_student_performance() -> Markdown:
    student_performance_dataset = StudentPeformanceDataset()
    return data_analysis(student_performance_dataset)

## Step 12: Triggering the Data Analysis:

- Calls data analysis on both the student performance and books datasets.

In [ ]:
data_analysis_on_student_performance()
data_analysis_on_books()

---

# Thank You for visiting The Hackers Playbook! 🌐

If you liked this research material;

- [Subscribe to our newsletter.](https://thehackersplaybook.substack.com)

- [Follow us on LinkedIn.](https://www.linkedin.com/company/the-hackers-playbook/)

- [Leave a star on our GitHub.](https://www.github.com/thehackersplaybook)

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>
